In [1]:
import csv
import pandas as pd
import glob

friday = pd.read_csv('Friday Glasto.csv')
saturday = pd.read_csv('Saturday Glasto.csv')
sunday = pd.read_csv('Sunday Glasto.csv')



In [2]:
friday.Start = friday['Start'].replace(':', '', regex=True)
friday.Start = pd.to_numeric(friday.Start)
friday.Finish = friday['Finish'].replace(':', '', regex=True)
friday.Finish = pd.to_numeric(friday.Finish)
friday['Slot'] = friday['Start'].apply(lambda x: (x < 400 and 'fri_night') or (x < 1400 and 'fri_morn') or (x < 1830 and 'fri_aft') or (x < 2129 and 'fri_eve') or (x > 2129 and 'fri_night'))
friday.dtypes

Band      object
Start      int64
Finish     int64
Stage     object
Slot      object
dtype: object

In [3]:
saturday.Start = pd.to_numeric(saturday['Start'].replace(':', '', regex=True))
sunday.Start = pd.to_numeric(sunday['Start'].replace(':', '', regex=True))
saturday.Finish = pd.to_numeric(saturday['Finish'].replace(':', '', regex=True))
sunday.Finish = pd.to_numeric(sunday['Finish'].replace(':', '', regex=True))
saturday['Slot'] = saturday['Start'].apply(lambda x: (x < 400 and 'sat_night') or (x < 1400 and 'sat_morn') or (x < 1830 and 'sat_aft') or (x < 2129 and 'sat_eve') or (x > 2129 and 'sat_night'))
sunday['Slot'] = sunday['Start'].apply(lambda x: (x < 400 and 'sun_night') or (x < 1400 and 'sun_morn') or (x < 1830 and 'sun_aft') or (x < 2129 and 'sun_eve') or (x > 2129 and 'sun_night'))


In [4]:
def clashes(band, day):
    start = int(day[day.Band == band.upper()].Start)
    end = int(day[day.Band == band.upper()].Finish)
    if (start < 900):
        clash = day[((day.Start>start+2245) & (day.Start<end+2430))]
    elif end < 900:
        clash = day[((day.Start>start-115))]
    else:
        clash = day[((day.Start>start-115)&(day.Start<end+60))]
    return clash
clashes('Carly Rae Jepsen', friday)

,Band,Start,Finish,Stage,Slot
3,TEXAS,1615,1715,Pyramid,fri_aft
4,STEFFLON DON,1445,1545,Pyramid,fri_aft
11,CARLY RAE JEPSEN,1545,1645,Other,fri_aft
19,PALE WAVES,1630,1730,Woodsies,fri_aft
20,DIGGA D,1515,1600,Woodsies,fri_aft
28,LOUIS COLE,1600,1700,West Holts,fri_aft
36,UNKNOWN MORTAL ORCHESTRA,1645,1745,Park,fri_aft
37,BILLY NOMATES,1515,1615,Park,fri_aft


The clashes function checks the start and finish times of a chosen band. If the start time or end time is under 900 (which means it is in the early hours of the morning).
For acts that are not near midnight, the function lists the acts that start between hour and a quarter before the start and half an hour after the end of the chosen band.

In [5]:
clashes('THE CHEMICAL BROTHERS (DJ SET)', friday)

,Band,Start,Finish,Stage,Slot
0,ARCTIC MONKEYS,2215,2345,Pyramid,fri_night
7,WIZKID,2230,2345,Other,fri_night
15,HOT CHIP,2230,2345,Woodsies,fri_night
24,KELIS,2215,2345,West Holts,fri_night
32,FEVER RAY,2300,15,Park,fri_night
43,THE CHEMICAL BROTHERS (DJ SET),2230,30,Arcadia,fri_night
44,ELKKA,2130,2230,Arcadia,fri_night


In [6]:
clashes('SKEPTA B2B JAMMER', saturday)


,Band,Start,Finish,Stage,Slot
32,FATBOY SLIM,2300,15,Park,sat_night
44,MK,2300,0,Arcadia,sat_night


In [7]:
friday.describe

<bound method NDFrame.describe of                                 Band  Start  Finish       Stage       Slot
0                     ARCTIC MONKEYS   2215    2345     Pyramid  fri_night
1                        ROYAL BLOOD   2015    2115     Pyramid    fri_eve
2                       THE CHURNUPS   1815    1930     Pyramid    fri_aft
3                              TEXAS   1615    1715     Pyramid    fri_aft
4                       STEFFLON DON   1445    1545     Pyramid    fri_aft
5                      MAISIE PETERS   1315    1415     Pyramid   fri_morn
6   THE MASTER MUSICIANS OF JOUJOUKA   1200    1245     Pyramid   fri_morn
7                             WIZKID   2230    2345       Other  fri_night
8                       FRED AGAIN..   2030    2130       Other    fri_eve
9                           CHVRCHES   1845    1945       Other    fri_eve
10                     KREPT & KONAN   1715    1815       Other    fri_aft
11                  CARLY RAE JEPSEN   1545    1645       Other   

In [8]:
alldays = pd.concat([friday, saturday, sunday])
alldays.Band = alldays['Band'].replace('\(', '', regex=True)
alldays.Band = alldays['Band'].replace('\)', '', regex=True)

alldays.head(50)

,Band,Start,Finish,Stage,Slot
0,ARCTIC MONKEYS,2215,2345,Pyramid,fri_night
1,ROYAL BLOOD,2015,2115,Pyramid,fri_eve
2,THE CHURNUPS,1815,1930,Pyramid,fri_aft
3,TEXAS,1615,1715,Pyramid,fri_aft
4,STEFFLON DON,1445,1545,Pyramid,fri_aft
5,MAISIE PETERS,1315,1415,Pyramid,fri_morn
6,THE MASTER MUSICIANS OF JOUJOUKA,1200,1245,Pyramid,fri_morn
7,WIZKID,2230,2345,Other,fri_night
8,FRED AGAIN..,2030,2130,Other,fri_eve
9,CHVRCHES,1845,1945,Other,fri_eve


In [14]:
def make_schedule(acts):
    fri_morn = ['Fri morn:']
    fri_aft = ['Fri aft:']
    fri_eve = ['Fri eve:']
    fri_night = ['Fri night:']
    sat_morn = ['Sat morn:']
    sat_aft = ['Sat aft:']
    sat_eve = ['Sat eve:']
    sat_night = ['Sat night:']
    sun_morn = ['Sun morn:']
    sun_aft = ['Sun aft:']
    sun_eve = ['Sun eve:']
    sun_night = ['Sun night:']
    string_lists = ['fri_morn', 'fri_aft', 'fri_eve','fri_night','sat_morn','sat_aft','sat_eve','sat_night','sun_morn','sun_aft','sun_eve','sun_night']
    list_lists = [fri_morn, fri_aft, fri_eve,fri_night,sat_morn,sat_aft,sat_eve,sat_night,sun_morn,sun_aft,sun_eve,sun_night]
    for act in acts:
        slot = alldays.Slot[(alldays.Band == act)]
        for i, v in enumerate(string_lists):
               if str(slot.iloc[0]) == v:
                    list_lists[i].append(act)
    return list_lists

acts_list = ['KELIS','FRED AGAIN..', 'THE HU', 'THE CHURNUPS', 'SOPHIE ELLIS-BEXTOR','GUNS N\' ROSES', 'MÅNESKIN', 'ARCTIC MONKEYS', 'SKEPTA B2B JAMMER', 'ELTON JOHN', 'TEXAS', 'LIZZO', 'LANA DEL REY', 'SHY FX FT STAMINA MC', 'THE CHEMICAL BROTHERS DJ SET', 'LEWIS CAPALDI', 'CARLY RAE JEPSEN', 'STEFFLON DON', 'YUSUF / CAT STEVENS', 'RAYE', 'RINA SAWAYAMA', 'FATBOY SLIM', 'BECKY HILL']

make_schedule(acts_list)

[['Fri morn:'],
 ['Fri aft:', 'THE CHURNUPS', 'TEXAS', 'CARLY RAE JEPSEN', 'STEFFLON DON'],
 ['Fri eve:', 'FRED AGAIN..'],
 ['Fri night:', 'KELIS', 'ARCTIC MONKEYS', 'THE CHEMICAL BROTHERS DJ SET'],
 ['Sat morn:', 'RAYE'],
 ['Sat aft:', 'LEWIS CAPALDI'],
 ['Sat eve:', 'MÅNESKIN', 'LIZZO', 'RINA SAWAYAMA'],
 ['Sat night:',
  "GUNS N' ROSES",
  'SKEPTA B2B JAMMER',
  'LANA DEL REY',
  'FATBOY SLIM'],
 ['Sun morn:', 'SOPHIE ELLIS-BEXTOR'],
 ['Sun aft:', 'THE HU', 'YUSUF / CAT STEVENS', 'BECKY HILL'],
 ['Sun eve:', 'ELTON JOHN'],
 ['Sun night:', 'SHY FX FT STAMINA MC']]

In [12]:
clashes('THE CHURNUPS', friday)

,Band,Start,Finish,Stage,Slot
2,THE CHURNUPS,1815,1930,Pyramid,fri_aft
9,CHVRCHES,1845,1945,Other,fri_eve
10,KREPT & KONAN,1715,1815,Other,fri_aft
17,TBA,1930,2030,Woodsies,fri_eve
18,COURTEENERS,1800,1900,Woodsies,fri_aft
26,JOEY BADASS,1900,2000,West Holts,fri_eve
27,GABRIELS,1730,1830,West Holts,fri_aft
34,SHYGIRL,1945,2045,Park,fri_eve
35,THE COMET IS COMING,1815,1915,Park,fri_aft
